In [ ]:
###############################################################################
###############################################################################
# Loops
###############################################################################
###############################################################################
https://medium.com/@rtjeannier/pandas-101-cont-9d061cb73bfc

In [ ]:
# Replace strings in Domainnames
# Just an incomplete example so the regex does not clean the data correctly
domain                              host                                 url
erotikchatz.de                      erotikchatz.de                       erotikchatz.de
erotiklife.de                       erotiklife.de                        erotiklife.de
erstberlin.de                       erstberlin.de                        erstberlin.de


nom = pd.read_json(r"D:\6_Rohdaten\Crawling\Nominet\nominet_domains.json", orient='record', encoding='utf_8', lines=False)
nomloop = nom.iloc[1:200:,0:3]
def myreplace(s):
    for ch in [r'www.', r'http', '://', "/"]:
        s = s.replace(ch, "")
    return s
nomloop["url"] = nomloop["url"].map(myreplace)


In [ ]:
###############################################################################
# This is an example how you cleaned data for the Naive Bayes
###############################################################################
#Testdata
review,label
The picture is clear and beautiful,positive
Picture is not clear,negative
#Traindata
review,label
Colors & clarity is superb,positive
Sadly the picture is not nearly as clear or bright as my 40 inch Samsung,negative
# This in Files
test = r"D:\Git\fakeshop_analysis\Scikit_NB\Example\test.txt"
train = r"D:\Git\fakeshop_analysis\Scikit_NB\Example\train.txt"

def load_data(filename):
    """ This function works, but you have to modify the second-to-last line from
    reviews.append(line[0].split()) to reviews.append(line[0]).
    CountVectorizer will perform the splits by itself as it sees fit, trust him :)"""
    reviews = list()
    labels = list()
    with open(filename) as file:
        file.readline()
        for line in file:
            line = line.strip().split(',')
            labels.append(line[1])
            reviews.append(line[0])
    return reviews, labels

X_train, y_train = load_data(train)
X_test, y_test   = load_data(train)

In [ ]:
###############################################################################
# REGEX vs ahocorastick Algorithm search: 
###############################################################################
https://stackoverflow.com/questions/48541444/pandas-filtering-for-multiple-substrings-in-series/48600345#48600345
https://pyahocorasick.readthedocs.io/en/latest/

import numpy as np
import random
import pandas as pd
import ahocorasick
import re

random.seed(321)

def orig(col, lst):
    mask = np.logical_or.reduce([col.str.contains(i, regex=False, case=False) 
                                 for i in lst])
    return mask

def using_regex(col, lst):
    """https://stackoverflow.com/a/48590850/190597 (Alex Riley)"""
    esc_lst = [re.escape(s) for s in lst]
    pattern = '|'.join(esc_lst)
    mask = col.str.contains(pattern, case=False)
    return mask

def using_ahocorasick(col, lst):
    A = ahocorasick.Automaton(ahocorasick.STORE_INTS)
    for word in lst:
        A.add_word(word.lower())
    A.make_automaton() 
    col = col.str.lower()
    mask = col.apply(lambda x: bool(list(A.iter(x))))
    return mask

N = 50000
# 100 substrings of 5 characters
lst = [''.join([chr(random.randint(0, 256)) for _ in range(5)]) for _ in range(100)]

# N strings of 20 characters
strings = [''.join([chr(random.randint(0, 256)) for _ in range(20)]) for _ in range(N)]
# make about 10% of the strings match a string from lst; this helps check that our method works
strings = [_ if random.randint(0, 99) < 10 else _+random.choice(lst) for _ in strings]

col = pd.Series(strings)

expected = orig(col, lst)
for name, result in [('using_regex', using_regex(col, lst)),
                     ('using_ahocorasick', using_ahocorasick(col, lst))]:
    status = 'pass' if np.allclose(expected, result) else 'fail'
    print('{}: {}'.format(name, status))



In [ ]:
###############################################################################
# Parse Datetes using this package in a loop
###############################################################################
from dateutil.parser import parse

date_array = [
    '2018-06-29 08:15:27.243860',
    'Jun 28 2018  7:40AM',
    'Jun 28 2018 at 7:40AM',
    'September 18, 2017, 22:19:55',
    'Sun, 05/12/1999, 12:30PM',
    'Mon, 21 March, 2015',
    '2018-03-12T10:12:45Z',
    '2018-06-29 17:08:00.586525+00:00',
    '2018-06-29 17:08:00.586525+05:00',
    'Tuesday , 6th September, 2017 at 4:30pm'
]

for date in date_array:
    print('Parsing: ' + date)
    dt = parse(date)
    print(dt.date())
    print(dt.time())
    print(dt.tzinfo)
    print('\n')


In [ ]:
###############################################################################
# Multiply a list of numbes with one singel number 
###############################################################################
# Loopy loop
list = [1,2,3,4,5,6,7,8,9]
my_new_list = []
for i in list:
    my_new_list.append(i * (24*60*60))

# Or the same with a list comprehension
my_list = [1, 2, 3, 4, 5]
my_new_list = [i * 5 for i in my_list]

In [ ]:
###############################################################################
# Function to count NA's in a Dataframe
###############################################################################
#https://stackoverflow.com/questions/26266362/how-to-count-the-nan-values-in-a-column-in-pandas-dataframe
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [ ]:
###############################################################################
# Split
# https://www.datacamp.com/community/tutorials/categorical-data
###############################################################################
dummy_df_age = pd.DataFrame({'age': ['0-20', '20-40', '40-60','60-80']})
dummy_df_age['start'], dummy_df_age['end'] = zip(*dummy_df_age['age'].map(lambda x: x.split('-')))

dummy_df_age.head()

In [ ]:
###############################################################################
# Replace a range with its mean
# https://www.datacamp.com/community/tutorials/categorical-data
###############################################################################
dummy_df_age = pd.DataFrame({'age': ['0-20', '20-40', '40-60','60-80']})

def split_mean(x):
    split_list = x.split('-')
    mean = (float(split_list[0])+float(split_list[1]))/2
    return mean

dummy_df_age['age_mean'] = dummy_df_age['age'].apply(lambda x: split_mean(x))

dummy_df_age.head()


In [ ]:
###############################################################################
# 
###############################################################################
dummy_df_age = pd.DataFrame({'age': ['0-20', '20-40', '40-60','60-80']})

def split_mean(x):
    split_list = x.split('-')
    mean = (float(split_list[0])+float(split_list[1]))/2
    return mean

dummy_df_age['age_mean'] = dummy_df_age['age'].apply(lambda x: split_mean(x))

dummy_df_age.head()


In [ ]:
###############################################################################
# Geron: Chapter3
###############################################################################
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]


In [ ]:
###############################################################################
# Geron Decsision Trees 
###############################################################################
a. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn's ShuffleSplit class for this.

from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))



In [ ]:
###############################################################################
# Identify langages
###############################################################################

from textblob import TextBlob
def detect_language(text):
    if len(text)>3:
        r=TextBlob(text)
        lang = r.detect_language()
        return lang
dfx['language']=dfx.html_body_3.apply(lambda x: detect_language(x))

In [ ]:
###############################################################################
# Write a function that constructs this vocabulary
###############################################################################
As an example, say that we were to use one-hot encoding for the following sentences:

s1. "This is sentence one."
s2. "Now, here is our sentence number two."

The vocabulary from the two sentences is:

vocabulary = {"here": 0, "is": 1, "now": 2, "number": 3, "one": 4, "our": 5, "sentence": 6, "this": 7, "two": 8}


In [ ]:
###############################################################################
# Write this nltk/mlp task
###############################################################################

Take a document as the input.
Read the document line by line
Tokenize the line
Stem the words
Output the stemmed words (print on screen or write to a file)
Repeat step 2 to step 5 until it is to the end of the document.


In [ ]:
###############################################################################
# Write this
###############################################################################
1. Take a couple of different documents like pdfs with different topics (Economics, Engineering, Flowers, ...)
2 Extract the text
3. Print the most frequent words per Topic


In [ ]:
###############################################################################
# Document Clustering
###############################################################################
Download the nltk.gutenberg books and perform a cluster analysis on them to see which books are similar
Do this after having read the chapter about clustering





In [ ]:
###############################################################################
# Practice the python .apply function
###############################################################################



In [ ]:
###############################################################################
# Figure out how to apply this to to a column
###############################################################################

import spacy
nlp = spacy.load('de_core_news_lg')

mails=['Hallo. Ich spielte am frühen Morgen und ging dann zu einem Freund. Auf Wiedersehen', 'Guten Tag Ich mochte Bälle und will etwas kaufen. Tschüss']

mails_lemma = []

for mail in mails:
     doc = nlp(mail)
     result = ''
     for token in doc:
        result += token.lemma_
        result += ' ' 
     mails_lemma.append(result)
print(mails_lemma)

In [ ]:
###############################################################################
# A small practice problem
###############################################################################
"""
Take this Loop and modify it by applying a function on those words like len
The result should be a DF with two columns:
C1: the words in thislist
C2: the length of each word

Try it in two different ways:
Way 2: Us e a DF and write the result to that DF

You can also vary the task by preprocessing more like: strip whitespace-> lowercase->length
"""

# Way 1: Use a list and write the results to a list
empty = []
thislist = ["apple", "banana", "cherry", "Bujaaa", "  Holy  Friholes! "]
for x in thislist:
    temp1 = x.lower()
    temp2 = temp1.replace(' ', '')
    temp3 = len(temp2)
    empty.append(temp3)
    print(temp3) 


# Way 2: Us e a DF and write the result to that DF

In [ ]:
###############################################################################
# A small practice problem
###############################################################################
# Split the items in two parts
# Save them in separate lists
# Count the number of characters in each

thislist = ["AA 123", "AB 1234", "AC 12345", "AD 123456", "AE 1234567"]

In [ ]:
    
###############################################################################
# A small practice problem
###############################################################################
# Take a df which contains text
# write a loop which goes trough the text and looks for words
# Every new word found is added to a list
# The list should contain all unique words which can be found in the text

In [ ]:
###############################################################################
# A small practice problem
###############################################################################

###############################################################################
# A small practice problem
###############################################################################

###############################################################################
# A small practice problem
###############################################################################

###############################################################################
# A small practice problem
###############################################################################













In [5]:
###############################################################################
# A small practice problem
###############################################################################

list1 = ["geeks", "for", "geeks"]
list2 = ("geeks", "for", "geeks", "squared")
string1 = "test"

for i in string1:
    print(i.upper())

T
E
S
T


In [8]:
for letter in "geeksforgeeks":
    if letter == "e" or letter =="s":
        continue
    print("current letter :", letter)

current letter : g
current letter : k
current letter : f
current letter : o
current letter : r
current letter : g
current letter : k


In [9]:
geeks = "geeksforgeeks"
for letter in geeks:
    if letter=="i" or letter=="g":
        continue
    print("current letter: ", letter)

current letter:  e
current letter:  e
current letter:  k
current letter:  s
current letter:  f
current letter:  o
current letter:  r
current letter:  e
current letter:  e
current letter:  k
current letter:  s


In [14]:
geeks = "geeksforgeeks"
for letter in geeks:
    if letter=="z" or letter=="z":
        break
    if letter=="g":
        raise Exception ("No Way!")
    print("Current Letter:", letter)

Exception: No Way!

In [18]:
columnnames = ["age", "gender", "height"]
def column_name_changer(names):
    for i in names:
        print(i+"_new")

column_name_changer(columnnames)

age_new
gender_new
height_new


In [30]:
# Load Data
import pandas as pd
path = "/Users/largo/Library/Mobile Documents/com~apple~CloudDocs/DataScience/Case_Studies/Titanic_ML_Disaster/test.csv"
titanic = pd.read_csv(path, sep=",", header=0)
#data.columns = ["A", "B", "C", "D", "E"]
print(titanic.columns)
titanic.head()

# Renaming Function
def column_renamer(dataframe):
    names = dataframe.columns
    new = []
    for i in names:
        new.append(i + "_new")
    return new
        
#titanic.columns = column_renamer(titanic)
#titanic.head()

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


AttributeError: 'DataFrame' object has no attribute 'withColumnRenamed'

In [37]:
# Write a Function that splits the data by "Pclass_new" and writes a separate file for each
# Output the datasets into three datasets inside python: df1, df2, df3 = function(titanic)
"""
This function takes a dataframe and separates tit depending on a columns unique values
"""
from collections import namedtuple

def separator(dataset):
    file1 = []
    file2 = []
    file3 = []
    file1 = dataset.loc[titanic.Pclass==1, :]
    file2 = dataset.loc[titanic.Pclass==2, :]
    file3 = dataset.loc[titanic.Pclass==3, :]
    Desc = namedtuple("Desc", ["test1", "test2", "test3"])
    return Desc(
        file1,
        file2,
        file3
    )

df1, df2, df3 = separator(titanic)

In [79]:
"""
    Identify the unique values inside a column
use each to split the dataframe
put the dataframes in a list?
loop over that list and put them in separate dataframes
Output should be two tings: list with df's AND list with the names od the df's
"""
def column_separator(dataset, column):
    data_list = []
    unique_values = set(dataset.column)
    for i in unique_values:
        data_list.append([dataset.loc[dataset.column==i, :]])
    return data_list

result_list = column_separator(titanic, titanic.Pclass)




AttributeError: 'DataFrame' object has no attribute 'column'